In [1]:
import json
with open('transfer_results.json', 'r') as fp:
    tx_data = json.load(fp)

In [2]:
import pandas as pd
df = pd.DataFrame(tx_data)
df['mint'] = df['from'] == '0x0000000000000000000000000000000000000000'
df.columns

Index(['from', 'to', 'value', 'transferIndex', 'borrowCap', 'supplyCap',
       'totalVariableDebt', 'variableBorrowRate', 'blockNumber',
       'blockTimestamp', 'transactionHash', 'mint'],
      dtype='object')

In [3]:
minters = df[df['mint']]['to']
len(minters)

7646

## First order flows

### Hoarders

In [4]:
senders = set(df['from'])
hoarders = [user for user in minters if user not in senders]
len(set(hoarders))

141

### Contracts

In [10]:
minter_txs = df[df['from'].isin(minters)]
#minter_txs = minter_txs.drop_duplicates(['from', 'to'])

In [7]:
import os
import requests

API_KEY = os.environ.get('ETHERSCAN_TOKEN')
contract_names = {'0x00907f9921424583e7ffbfedf84f92b7b2be4977': 'GhoAToken',
 '0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d': 'UniswapV3Pool',
 '0x1a88df1cfe15af22b3c4c783d4e6f7f9e0c1885d': 'GHO Staking',
 '0x9008d19f58aabd9ed0d60971565aa8510560ab41': 'CowSwap',
 '0xdef171fe48cf0115b1d80b88dc8eab59176fee57': 'ParaSwap',
 '0x383e7acd889bf57b0d79a584009cb570534ab518': 'UniswapV3Pool',
 '0xe37e799d5077682fa0a244d46e5649f71457bd09': '1inch',
 '0x54eebc36527fe2e5624051e3c895810d7b68bcfc': 'UniswapV3Pool',
 '0xba12222222228d8ba445958a75a0704d566bf2c8': 'Balancer',
 '0x74de5d4fcbf63e00296fd95d33236b9794016631': 'AirSwap',
 '0xad3b67bca8935cb510c8d18bd45f0b94f54a968f': '1inch',
 '0x8d8404f8cca4c8834ca3cab1e54887ae47724bee': '1inch',
 '0x5756880b6a1eaba0175227bf02a7e87c1e02b28c': 'GHO CCIP',
 '0x22f9dcf4647084d6c31b2765f6910cd85c178c18': '0x Exchange',
 '0xbbec426df87537ad0bec0e899007be441427d0f8': 'UniswapV3Pool',
 '0xf0d4c12a5768d806021f80a262b4d39d26c58b8d': 'Curve',
 '0x3208684f96458c540eb08f6f01b9e9afb2b7d4f0': '1inch',
 '0x92f3f71cef740ed5784874b8c70ff87ecdf33588': '1inch',
 '0xf081470f5c6fbccf48cc4e5b82dd926409dcdd67': 'Kyber Network',
 '0x6f42b83ecda76a8313fd8a45ca18a3fdfd37bbc7': 'Gearbox',
 '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae': 'Li.Fi',
 '0x14cf6d2fe3e1b326114b07d22a6f6bb59e346c67': 'Maverick',
 '0x16c6521dff6bab339122a0fe25a9116693265353': 'Curve',
 '0xf4d34d7fdd2ed5d57c6f583237bb038fc3839d17': '1inch',
 '0x9a83b11e50ff5cc544f1401f35807b7c07de791d': 'UniswapV3Pool',
 '0x28104d4f703ee5b5011cefe106f54efd56f33f95': 'Odos',
 '0x5f0000d4780a00d2dce0a00004000800cb0e5041': 'ParaSwap',
 '0x670a72e6d22b0956c0d2573288f82dcc5d6e3a61': 'Curve',
 '0xbb33ed8c1148e62411a97d25390fd48698fd937e': '1inch',
 '0xd8533305aa8d9eb5681f15f1461e02c1601eed72': 'UniswapV3Pool',
 '0x5f515f6c524b18ca30f7783fb58dd4be2e9904ec': '1inch',
 '0x050ebe3dbb4b3a3526735b04cc3d96c80609ee7e': 'Maverick',
 '0x86152df0a0e321afb3b0b9c4deb813184f365ada': 'Curve',
 '0xbf683b7aaf9accea6bff8e926e03f6ceb98b525a': '1inch',
 '0xd1742b3c4fbb096990c8950fa635aec75b30781a': '1inch',
 '0x74345504eaea3d9408fc69ae7eb2d14095643c5b': 'Curve',
 '0xc8c9cfad493cffffd6684da0f45d141e33b6ef27': '1inch',
 '0x0d67b27c340887da87344c344c20ce60a8eb84c5': 'Maverick',
 '0x99a58482bd75cbab83b27ec03ca68ff489b5788f': 'Curve',
 '0xa8376f53391a041c8236a232f7f019ea76eed86d': 'Odos',
 '0x1a22c1886196101babcac1bf4223294ab04c1d66': 'Odos',
 '0xc48c6d400de27081df85d55100c7b06f190c764a': 'Maverick',
 '0x9f274d28fedce05153f9c810b8070277eaf8c031': 'Maverick',
 '0x635ef0056a597d13863b73825cca297236578595': 'Curve',
 '0x0b8a49d816cc709b6eadb09498030ae3416b66dc': '1inch',
 '0x1d7405df25fd2fe80390da3a696dcfd5120ca9ce': 'ParaSwap',
 '0x8dd418fa80b7f37856e36199e67013f00e5d7a61': 'Maverick',
 '0xd08d0006f00040b400180f9500b00c5026ac0900': 'ParaSwap',
 '0x19ea2e6f21bdfc894abf09fa179d59f6c0e0797b': '1inch',
 '0x63e988f4b30245f9f4ee898531e0be3fee20b170': '1inch',
                 #########
  '0x686f8d21520f4ecec7ba577be08354f4d1eb8262': 'GHO Stability Module',
                  '0x0d8effc11df3f229aa1ea0509bc9dfa632a13578': 'GHO Stability Module',
                  '0x0cbc13b84286007ec4f17cd0f9dc7d7ca6ee4fff': 'Odos',
                  '0x7d2b63a9ab475397d9c247468803f25cf6523b76': 'Odos',
                  '0xc351e45db65d68585e180795537563d33b3716e7': 'ParaSwap',
                  '0x9c211ec76380bae8e370a517df95075b40d34cdf': 'Maverick',
                  '0x005c78a48b482c1733c7cf958e65d90d3d40554b': 'ParaSwap',
                  '0x9c594c2e2e2e5aa300be12596215188c324c3e7c': 'Pendle',
                  '0x3d20601ac0ba9cae4564ddf7870825c505b69f1a': 'Pendle',
                  '0xe4e9278aed2482248979768a71ca4bfd0c999988': "MultiSigs",
                  '0xe705b1d26b85c9f9f91a3690079d336295f14f08': "MultiSigs",
                  '0x205e795336610f5131be52f09218af19f0f3ec60': "MultiSigs",
                  "0xd211a02a0adde56bb7f9700f49d4ba832adc7ddf": "MultiSigs",
                  "0xe8d4d93d9728bd673b0197673a230f62255c7846": "MultiSigs",
                  '0x6e7058c91f85e0f6db4fc9da2ca41241f5e4263f': "Notional",
                  '0xff98eb65eb5a727d4354d37abb3ef27537a56b60': "DeFi Saver",
                  '0xc2b92b6c69e5c3b6b29385c1a17fee906e0ca910': "DeFi Saver",
                  '0x8c82d963eb282cb7a751e551e3997ba66c2c8237': "DeFi Saver",
                  "0xb639d208bcf0589d54fac24e655c79ec529762b8": "DeFi Saver",
                  "0x0e18476415d63882f5a82ad0ecae0072e42ce992": "DeFi Saver",
                  "0xbb67b81dd080a406227a38965d0393f396ddecbc": "DeFi Saver",
                  "0x02dcc3438bb7e46f6bf671c72e98d503c0bce520": "DeFi Saver",
                  "0x3f64bc83f10c4f11a2c7fbd9190949b1c8d54a43": "DeFi Saver",
                  "0xe27baebd7b14602de3797974db9f5f4f8dcb6679": "MEV Bots",
                  "0xa87f4f8b87abf5237c16971e406eef0e2d541c9e": "MEV Bots",
                  "0xd5e74de4385ef0eb1bb4db05a6a504f04d92e79d": "MEV Bots",
                  "0xe20cd9377c204a27952f8b41075f0b8bd1ceec3d": "MEV Bots",
                  "0xf1d2d880c5dde7cc912636c2b4d080b3fe5f7b50": "MEV Bots",
                  "0x00000000009e50a7ddb7a7b0e2ee6604fd120e49": "MEV Bots",
                  "0x2593154f192f46f65a4a0315a5d77556d5d3d264": "MEV Bots",
                  "0x0000000000000000000000000000000000000000": "Burn",
                  
                 }

In [8]:
for contract in minter_txs['to'].value_counts()[:60].index:
    if contract in contract_names: 
        continue
    URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract}&apikey={API_KEY}"
    r = requests.get(URL)
    if r.json()['result'][0]['Implementation']:
        impl_contract = r.json()['result'][0]['Implementation']
        URL = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={impl_contract}&apikey={API_KEY}"
        r = requests.get(URL)        
    #contract_names[contract] = (r.json()['result'][0]['ContractName'])
    print(contract, (r.json()['result'][0]['ContractName']))

0x464c71f6c2f760dda6093dcb91c24c39e5d6e18c Collector
0x6467e807db1e71b9ef04e0e3afb962e4b0900b2b FeeReceiver
0x329c54289ff5d6b7b7dae13592c6b1eda1543ed4 
0x0f4a1d7fdf4890be35e71f3e0bbc4a0ec377eca3 
0xa9c0cded336699547aac4f9de5a11ada979bc59a 
0x55877bd7f2ee37bde55ca4b271a3631f3a7ef121 
0x93ebc3ca85f96afd72edb914e833fe18888de179 Yearn V3 Vault


In [11]:
minter_txs.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts()

GhoAToken               3054
UniswapV3Pool           2574
ParaSwap                1514
GHO Staking             1317
CowSwap                 1229
Others                  1100
1inch                    922
DeFi Saver               852
Curve                    289
Balancer                 260
AirSwap                  212
Maverick                 189
MEV Bots                  81
0x Exchange               79
GHO CCIP                  70
Odos                      64
Li.Fi                     62
Kyber Network             58
Gearbox                   56
Burn                      42
Pendle                    26
Notional                  21
MultiSigs                 18
GHO Stability Module      15
Name: to, dtype: int64

### Sent to EOAs

In [13]:
other_addresses = [user for user in minter_txs['to'] if user not in contract_names]
len(set(other_addresses))

411

In [14]:
import asyncio
from web3 import Web3
import concurrent
from tqdm import tqdm

provider = os.environ.get('WEB3_PROVIDER_URL')
w3 = Web3(Web3.HTTPProvider(provider))

def check_address_type(address):
    code = w3.eth.get_code(Web3.to_checksum_address(address))
    return address, len(code) > 0

def process_batch(addresses):
    return [check_address_type(addr) for addr in addresses]

def parallel_address_check(addresses, batch_size=20):
    eoa_count = 0
    contract_count = 0
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        batches = [addresses[i:i + batch_size] for i in range(0, len(addresses), batch_size)]
        futures = [executor.submit(process_batch, batch) for batch in batches]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            batch_results = future.result()
            for address, is_contract in batch_results:
                if is_contract:
                    contract_count += 1
                else:
                    eoa_count += 1
                results.append((address, 'Contract' if is_contract else 'EOA'))

    return results, eoa_count, contract_count

results, eoa_count, contract_count = parallel_address_check(list(set(other_addresses)))

print(f"EOAs: {eoa_count}")
print(f"Contracts: {contract_count}")

100%|████████████████████████████████████████████████████████████████████████████████| 21/21 [00:12<00:00,  1.69it/s]

EOAs: 299
Contracts: 112


In [15]:
first_order_flows = minter_txs.copy()['to'].apply(lambda x: contract_names.get(x, "Others")).value_counts().to_dict()
first_order_flows['Sent to other EOA'] = eoa_count
first_order_flows["Others"] = contract_count
first_order_flows["Held in wallet"] = len(set(hoarders))
first_order_flows

{'GhoAToken': 3054,
 'UniswapV3Pool': 2574,
 'ParaSwap': 1514,
 'GHO Staking': 1317,
 'CowSwap': 1229,
 'Others': 112,
 '1inch': 922,
 'DeFi Saver': 852,
 'Curve': 289,
 'Balancer': 260,
 'AirSwap': 212,
 'Maverick': 189,
 'MEV Bots': 81,
 '0x Exchange': 79,
 'GHO CCIP': 70,
 'Odos': 64,
 'Li.Fi': 62,
 'Kyber Network': 58,
 'Gearbox': 56,
 'Burn': 42,
 'Pendle': 26,
 'Notional': 21,
 'MultiSigs': 18,
 'GHO Stability Module': 15,
 'Sent to other EOA': 299,
 'Held in wallet': 141}

## Second Order Flows

In [16]:
minter_full_txs = df[df['from'].isin(minters)]

We're only going to check DEX trades

In [17]:
dexes = ['1inch', 'Odos', 'Curve', 'CowSwap', 'ParaSwap', 'Maverick', 'Balancer', 'UniswapV3Pool', '0x Exchange', 'Kyber Network', 'Li.Fi', 'Others', 'MultiSigs', 'MEV Bots']
minter_full_txs['destination'] =  minter_full_txs['to'].apply(lambda x: contract_names.get(x, "Others"))
dex_txs = minter_full_txs[minter_full_txs['destination'].isin(dexes)]
dex_txs

/tmp/ipykernel_180125/1021720791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minter_full_txs['destination'] =  minter_full_txs['to'].apply(lambda x: contract_names.get(x, "Others"))


,from,to,value,transferIndex,borrowCap,supplyCap,totalVariableDebt,variableBorrowRate,blockNumber,blockTimestamp,transactionHash,mint,destination
10,0x79603115df2ba00659adc63192325cf104ca529c,0x5459ba81d4b375a7b2e34aa83edb1e18c4dc181e,155934142056500000000,10,0,0,81343918969610398129157,15000000000000000000000000,17699326,1689430739,0x9e71943828922409e045ff5c9b9011925d542de62a7d...,False,Others
18,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,28256317265407911651909,18,0,0,277080953174315786866668,15000000000000000000000000,17699363,1689431195,0xc2c13363f315249fc3481b0c94397edbc15269b87bad...,False,Others
38,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,16812655822481809110230,38,0,0,278461016597257517749073,15000000000000000000000000,17699401,1689431675,0x2f8ebe2e50c7b6ddbbaab901e68936d166aef8a01406...,False,Others
50,0x9e9fbeb970383a43d6fba10af4bc0a96e567f83a,0x8e5c57e69ff1a5a34a0358a260d73981f38e924c,100000000000000000000,50,0,0,281362106879226153725806,15000000000000000000000000,17699435,1689432107,0x8f0f230cabf7e21a954886310b70dd0b7c1da1799ff4...,False,Others
55,0x2e21f5d32841cf8c7da805185a041400bf15f21a,0xc0268fdbc7f526b43cdbf34e7529b7da0d06dd55,12977707562003048338489,55,0,0,281162188722374181974933,15000000000000000000000000,17699485,1689432719,0x65ad012bb84f96890b88a8cbc1d1aa889333bab85ef2...,False,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82022,0x0a9972b72910b8668755d0dc016ecc68d1cbbcca,0x16c6521dff6bab339122a0fe25a9116693265353,5000000000000000000000,82022,125000000,0,111471857694936162785433874,60000000000000000000000000,20682692,1725518867,0x84b13ede19fa90a3e9c92e514b89d13d54542d28ddeb...,False,Curve
82065,0xd6c1f6cd5134568979f3c8cf41ed2332cb6b4a74,0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae,2017588478833170819005,82065,125000000,0,111470318026089831961216830,60000000000000000000000000,20683212,1725525167,0x9427f8f1445d4931c5e76ff3dcce55b9639b3a63a897...,False,Li.Fi
82069,0x4f0a01badaa24f762cee620883f16c4460c06be0,0x28104d4f703ee5b5011cefe106f54efd56f33f95,167000999250672397414184,82069,125000000,0,111470450365358614119627153,60000000000000000000000000,20683263,1725525791,0x6d58e97b207f484b57026614a034aef0e1518069f03c...,False,Odos
82078,0x4f0a01badaa24f762cee620883f16c4460c06be0,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,130421901502327359231,82078,125000000,0,111470554709892853590148427,60000000000000000000000000,20683304,1725526283,0x8e8e60ab09cd4a9ffcebf296395599fb7e08d34c0da2...,False,UniswapV3Pool


### Parse token transfers
We're going to look for ERC20 transfers going TO the dumper address in the same transaction and we'll fetch the symbol for those

In [18]:
from web3.exceptions import ContractLogicError
import aiohttp
import pandas as pd

w3 = Web3(Web3.HTTPProvider(provider))

ERC20_ABI = [
    {
        "constant": True,
        "inputs": [],
        "name": "symbol",
        "outputs": [{"name": "", "type": "string"}],
        "type": "function"
    },
    {
        "anonymous": False,
        "inputs": [
            {"indexed": True, "name": "from", "type": "address"},
            {"indexed": True, "name": "to", "type": "address"},
            {"indexed": False, "name": "value", "type": "uint256"}
        ],
        "name": "Transfer",
        "type": "event"
    }
]

LP_EVENT_TOPICS = [
    '0x3067048beee31b25b2f1681f88dac838c8bba36af25bfb2b7cf7473a5847e35f' # IncreaseLiquidity UniV3
    '0x26f55a85081d24974e85c6c00045d0f0453991e95873f52bff0d21af4079a768' # AddLiquidity Curve
]

symbols = {'0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2'.lower(): 'MKR'}

def get_token_symbol(token_address):
    contract = w3.eth.contract(address=token_address, abi=ERC20_ABI)
    try:
        symbol = contract.functions.symbol().call()
        return symbol
    except ContractLogicError:
        return "Unknown"

def process_transaction(tx_hash, target_address, destination):
    try:
        receipt = w3.eth.get_transaction_receipt(tx_hash)
    except Exception as e:
        print(e)
        return {}
    events = []
    
    for log in receipt.logs:
        if len(log['topics']) in [3,4] and log['topics'][0].hex() == Web3.keccak(text="Transfer(address,address,uint256)").hex():
            to_address = '0x' + log['topics'][2].hex()[-40:]
            if to_address.lower() == target_address.lower():
                from_address = '0x' + log['topics'][1].hex()[-40:]
                token_address = log['address']
                
                if token_address.lower() in symbols:
                    symbol = symbols[token_address.lower()]
                else:
                    try:
                        symbol = get_token_symbol(token_address)
                        symbols[token_address.lower()] = symbol
                    except Exception as e:
                        print(f'{e}: {token_address} : {tx_hash}')
                        symbol = "Missing"
                
                events.append({
                    'tx_hash': tx_hash,
                    'token_address': token_address,
                    'symbol': symbol,
                    'from': from_address,
                    'to': to_address,
                    'destination': destination,
                })
    
    return events

def process_batch(batch):
    results = []
    for tx_hash, target_address, destination in batch:
        results.extend(process_transaction(tx_hash, target_address, destination))
    return results

def parallel_transaction_processing(tx_hashes, target_addresses, destinations, batch_size=20):
    all_events = []

    tx_address_pairs = list(zip(tx_hashes, target_addresses, destinations))

    with concurrent.futures.ThreadPoolExecutor() as executor:
        batches = [tx_address_pairs[i:i + batch_size] for i in range(0, len(tx_address_pairs), batch_size)]
        futures = [executor.submit(process_batch, batch) for batch in batches]

        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            batch_events = future.result()
            all_events.extend(batch_events)

    return pd.DataFrame(all_events)

tx_hashes = dex_txs['transactionHash'].to_list()
target_addresses = dex_txs['from'].to_list()
destinations = dex_txs['destination'].to_list()

result_df = parallel_transaction_processing(tx_hashes, target_addresses, destinations)

  0%|                                                                                        | 0/422 [00:00<?, ?it/s]

Transaction with hash: '0x4fd1714cd20f473946c229f8b38a363836c8df85d70a2c50b42179fe13a22351' not found.
Transaction with hash: '0x0dae1bf055cab21bf4d32cb247bc36262679dcc99c1200b9d71d7ff60ba109f3' not found.


  0%|▏                                                                               | 1/422 [00:03<25:00,  3.56s/it]

Transaction with hash: '0x91672b3409231ad7251d61f06f3241f3e50bce4220e97830925e180c137eb5ca' not found.


  2%|█▊                                                                             | 10/422 [00:04<01:26,  4.77it/s]

...

 67%|████████████████████████████████████████████████████▎                         | 283/422 [01:25<00:37,  3.72it/s]

Transaction with hash: '0x437919ef5b53afe61436b78dc04bff54dd7359303fb9fb4c0c8c471b5ca2d41e' not found.


 69%|██████████████████████████████████████████████████████▏                       | 293/422 [01:29<00:40,  3.17it/s]

Transaction with hash: '0x15fd72aaf66de77c30410a1ddee2466fc9db964d043c926b5873453816931c72' not found.


100%|██████████████████████████████████████████████████████████████████████████████| 422/422 [02:07<00:00,  3.32it/s]


In [35]:
pd.set_option('display.max_colwidth', None)

In [106]:
result_df

,tx_hash,token_address,symbol,from,to,destination
0,0x0d73f08d46ffd3f12f197a4e03eab32b4ab69457d11d...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xd14f076044414c255d2e82cceb1cb00fb1bba64c,CowSwap
1,0x850d108cf9568bb0a9855b81b746a4e00027e2e83585...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xd14f076044414c255d2e82cceb1cb00fb1bba64c,CowSwap
2,0x84b396c7fad17e16bb22ede6978340d240a1d61c8f93...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
3,0x2cfac29f66bffd5bac296c8b832e631589e2e185e277...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
4,0xc2c13363f315249fc3481b0c94397edbc15269b87bad...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,UNI-V3-POS,0x0000000000000000000000000000000000000000,0xd2eeff73117c86c14f11a6052620848f8dd6e0c8,Others
...,...,...,...,...,...,...
13211,0x2c4140ca025ba41e0d5a28bb483f6101c310beefbddd...,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,WBTC,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x97e1e6c70fe9f2209792640336534bfc9f93aa7a,UniswapV3Pool
13212,0xd6f7e7e0fbc67557e766698fe936040ec5771d342df1...,0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599,WBTC,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x97e1e6c70fe9f2209792640336534bfc9f93aa7a,UniswapV3Pool
13213,0xd3e459dcd796c526f22cea607844229cfc3d9b536b8e...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x9008d19f58aabd9ed0d60971565aa8510560ab41,0xe705b1d26b85c9f9f91a3690079d336295f14f08,CowSwap
13214,0x264dbb0f1eda1d684a1b70d26f8f6ec82093d6445172...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x5b93d80da1a359340d1f339fb574bdc56763f995,0x2767edfee9f5ba743511ee4187a596b302b43938,UniswapV3Pool


In [19]:
result_df['symbol'].value_counts()[:20]

GHO                   2943
USDC                  2187
USDT                  1025
WBTC                   439
DAI                    437
AAVE                   327
variableDebtEthGHO     278
WETH                   243
wstETH                 225
LINK                   157
UNI-V3-POS             108
GHO/bb-a-USD            77
aEthWETH                73
LUSD                    72
GP                      64
aEthwstETH              57
rETH                    52
crvUSD                  51
GHO/USDT/USDC           48
USDe                    42
Name: symbol, dtype: int64

In [41]:
dex_txs[dex_txs['transactionHash'] == '0x9d1efc90e836150b21ada9a2cebff14a1d47d74331a6bd5e67530c2cf112152e']

,from,to,value,transferIndex,borrowCap,supplyCap,totalVariableDebt,variableBorrowRate,blockNumber,blockTimestamp,transactionHash,mint,destination
79156,0x686f8d21520f4ecec7ba577be08354f4d1eb8262,0x9008d19f58aabd9ed0d60971565aa8510560ab41,691779724000000000000,79156,116000000,0,106694969088181240637862378,60000000000000000000000000,20641913,1725027179,0x9d1efc90e836150b21ada9a2cebff14a1d47d74331a6bd5e67530c2cf112152e,False,CowSwap


In [20]:
result_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()

to
destination   symbol                   
0x Exchange   AAVE                    7
              BTRFLY                  1
              DAI                     3
              DMT                     1
              GHO                     2
...                                  ..
UniswapV3Pool swETH                   1
              variableDebtEthGHO      4
              variableDebtEthcrvUSD   1
              wTAO                    2
              wstETH                 51

[485 rows x 1 columns]

In [21]:
grouped = result_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()
grouped = grouped.rename(columns={'to': 'count'})
grouped['percentage'] = grouped.groupby(level=0).apply(lambda x: x / x.sum() * 100)
grouped = grouped.reset_index()
grouped = grouped.set_index(['destination', 'symbol'])
grouped

/tmp/ipykernel_180125/3331406645.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  grouped['percentage'] = grouped.groupby(level=0).apply(lambda x: x / x.sum() * 100)


count  percentage
destination   symbol                                  
0x Exchange   AAVE                       7   10.606061
              BTRFLY                     1    1.515152
              DAI                        3    4.545455
              DMT                        1    1.515152
              GHO                        2    3.030303
...                                    ...         ...
UniswapV3Pool swETH                      1    0.051256
              variableDebtEthGHO         4    0.205023
              variableDebtEthcrvUSD      1    0.051256
              wTAO                       2    0.102512
              wstETH                    51    2.614044

[485 rows x 2 columns]

In [22]:
tokens_df = result_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()

In [23]:
tokens_df.loc['UniswapV3Pool']

,to
symbol,
AAVE,123
ACX,4
ANKR,1
ARB,2
ARES,1
...,...
swETH,1
variableDebtEthGHO,4
variableDebtEthcrvUSD,1


In [24]:
grouped.loc['Curve']

,count,percentage
symbol,,
AAVE,1,0.442478
CRV,2,0.884956
DAI,5,2.212389
DOLA,1,0.442478
ETHx,1,0.442478
FRAX,3,1.327434
FXS,1,0.442478
GHO,2,0.884956
GHOUSDe,4,1.769912


In [25]:
set(result_df['symbol'][result_df['symbol'].str.lower().str.startswith('var')].to_list())

{'variableDebtDAI',
 'variableDebtEthDAI',
 'variableDebtEthGHO',
 'variableDebtEthLDO',
 'variableDebtEthUSDC',
 'variableDebtEthUSDT',
 'variableDebtEthcrvUSD'}

In [26]:
LP_SYMBOLS = {'UNI-V3-POS','ECLP-GHO-GYD',
 'ECLP-GHO-USDC',
              'GHOBTCwstE',
 'ECLP-GHO-USDC-2',
 'GHO-3POOL-BPT',
 'GHO/bb-a-USD',
              'auraGHO/USDT/USDC-vault',
 'MBP-GHO-USDC-15-R1',
 'MBP-GHO-USDC-18-R1',
 'MBP-GHO-USDC-21',
 'MBP-GHO-USDC-21-R1',
              'SPT-PT/IBT-f',
 'MBP-GHO-USDC-23-R1',
 'MBP-GHO-USDC-4-R1',
 'UNI-V3-POS',
 'USDC-DAI-USDT',
 'bb-a-USD',
 'crvUSDGHO-f',
              'GHO/bb-a-USD',  'GHO/LUSD', 'GHO/USDT/USDC', 'GHO/bb-a-USD', 'GHO/USDT/USDC', 'GHOUSDe', 'fxUSDGHO', 'crvUSDGHO-f', 'GHOcrvUSD', 'GHOUSDe', '80wstETH/20GHO'}

ETH_AND_LSTS = {'boxETH',
 'cbETH',
 'ezETH',
 'osETH',
 'rETH',
 'ETH2X',
 'sfrxETH',
 'stETH',
 'swETH',
 'ETHx',
 'WETH',
 'weETH',
 'wstETH'}

BTC = {'LBTC', 'WBTC', 'tBTC'}

OTHER_STABLES = { 'mkUSD',
 'rgUSD',
 'sUSDe', 
 'fxUSD',
                 'sUSD',
 'stUSD', 
 'bb-a-USD',
 'USDP', 
 'PYUSD',
 'DOLA',
 'crvUSD',
 'USD0',
 'USD3',
 'USDA',
                 'FRAX',
 'USDe',
 'GUSD',
                 'sDAI'
 }

A_TOKENS = {
 'aEthAAVE',
 'aEthDAI',
 'aEthUSDC',
    'aEthETHx',
 'aEthUSDT',
 'aEthUSDe',
 'aEthWBTC',
 'aEthWETH',
 'aEthcrvUSD',
 'aEthrETH',
 'aEthwstETH'}

GHO = {'pGHO', 'YT-sw-stkGHO-1732492824', 'GHO-agg', 'nGHO', 'pdGHO', 'PT-sw-stkGHO-1732492824'}

VAR_DEBT_TOKENS = {'variableDebtEthGHO', 'variableDebtEthUSDC', 'variableDebtEthcrvUSD', 'variableDebtEthDAI'}

MAJORS = {'GHO', 'USDC', 'USDT', 'DAI', 'AAVE', 'LINK', 'LUSD'}


In [27]:
result_df[~result_df['symbol'].isin(VAR_DEBT_TOKENS | A_TOKENS | BTC | OTHER_STABLES | ETH_AND_LSTS | LP_SYMBOLS | MAJORS | GHO )]['symbol'].value_counts()[:50]

GP         64
MKR        34
CRV        32
stkAAVE    23
UNI        16
LDO        14
CVX        13
RPL        12
EURe       12
AURA       11
DIP        10
FXS         7
BITCOIN     7
OHM         7
TSUKA       6
SNX         6
RLB         6
PEPE        6
MPL         5
sdCRV       5
MATIC       5
BAL         5
wTAO        5
ILV         5
cvxCRV      4
SOL         4
R           4
ENA         4
PENDLE      4
RNDR        4
MPv2        4
ONDO        4
ACX         4
GNO         3
RIO         3
WOJAK       3
SHIB        3
PRISMA      2
ELA         2
EUL         2
ETHFI       2
ENS         2
MAV         2
PRIME       2
JPGD        2
GEAR        2
BLUR        2
VSP         2
GYD         2
ULTRA       2
Name: symbol, dtype: int64

### Relabelling our symbols

In [28]:
def relabel(label):
    if label in A_TOKENS:
        return 'aTokens'
    elif label in MAJORS:
        return label
    elif label in VAR_DEBT_TOKENS:
        return 'Variable Debt Tokens'
    elif label in OTHER_STABLES:
        return 'Other Stablecoins'
    elif label in BTC:
        return 'BTC'
    elif label in ETH_AND_LSTS:
        return 'ETH/LSTs/LRTs'
    elif label in GHO:
        return 'GHO-Yield Aggregation'
    elif label in LP_SYMBOLS:
        return 'Providing Liquidity'
    return 'Other Cryptos'

label_df = result_df.copy()
label_df['symbol'] = label_df['symbol'].apply(relabel)

label_df

,tx_hash,token_address,symbol,from,to,destination
0,0x84b396c7fad17e16bb22ede6978340d240a1d61c8f93...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Providing Liquidity,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
1,0x2cfac29f66bffd5bac296c8b832e631589e2e185e277...,0xC36442b4a4522E871399CD717aBDD847Ab11FE88,Providing Liquidity,0x0000000000000000000000000000000000000000,0x36c4bd54d54dd898c242f5f634f5d0cef3be2a8a,UniswapV3Pool
2,0x072ac24481a9a8220932a0f7f916593124203708314b...,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,USDC,0x5c95d4b1c3321cf898d25949f41d50be2db5bc1d,0xfb13776ac0a308919fc8a4eb9e8f7eac7d8f1bda,UniswapV3Pool
3,0x4e546baa664234ad58960ba1fd78fb2dc5be771ddd0b...,0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0,Other Cryptos,0x290a6a7460b308ee3f19023d2d00de604bcf5b42,0x698373ba1186edb9f9dd8ed46fb69f1580015153,UniswapV3Pool
4,0x97cf14deadbd3230a3394e9458f5c0565d1e3c4becfb...,0x6B175474E89094C44Da98b954EedeAC495271d0F,DAI,0x5777d92f208679db4b9778590fa3cab3ac9e2168,0x3cd361345df2a61d2fa0e2d1692da4404f1a4a11,UniswapV3Pool
...,...,...,...,...,...,...
9803,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344...,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,ETH/LSTs/LRTs,0xc351e45db65d68585e180795537563d33b3716e7,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9804,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344...,0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8,aTokens,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9805,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344...,0x786dBff3f1292ae8F92ea68Cf93c30b34B1ed04B,Variable Debt Tokens,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap
9806,0xb3c4f953af0c855cc35a8d350317135fa77376e1c344...,0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f,GHO,0x0000000000000000000000000000000000000000,0x60a4dadea54fd242d11462667598a73473543542,ParaSwap


In [29]:
label_df['symbol'].value_counts()

GHO                      2943
USDC                     2187
USDT                     1025
ETH/LSTs/LRTs             567
Other Cryptos             490
BTC                       443
DAI                       437
Providing Liquidity       385
AAVE                      327
Variable Debt Tokens      286
Other Stablecoins         244
aTokens                   228
LINK                      157
LUSD                       72
GHO-Yield Aggregation      17
Name: symbol, dtype: int64

In [30]:
grouped = label_df[['symbol', 'destination', 'to']].groupby(['destination', 'symbol']).count()
grouped = grouped.rename(columns={'to': 'count'})
grouped['percentage'] = grouped.groupby(level=0).apply(lambda x: x / x.sum() * 100)
grouped = grouped.reset_index()
grouped = grouped.set_index(['destination', 'symbol'])
grouped

/tmp/ipykernel_180125/376489346.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  grouped['percentage'] = grouped.groupby(level=0).apply(lambda x: x / x.sum() * 100)


count  percentage
destination   symbol                                 
0x Exchange   AAVE                      7   10.606061
              BTC                       6    9.090909
              DAI                       3    4.545455
              ETH/LSTs/LRTs             8   12.121212
              GHO                       2    3.030303
...                                   ...         ...
UniswapV3Pool Providing Liquidity     102    5.228088
              USDC                    715   36.647873
              USDT                    313   16.043055
              Variable Debt Tokens      5    0.256279
              aTokens                   5    0.256279

[156 rows x 2 columns]

## Sankey Chart

First let's reformat with a consistant format for 1st and 2nd order

In [31]:
first_order = [{'from': 'Minting', 'to': k, 'amount': v} for k, v in first_order_flows.items()]

In [32]:
first_order

[{'from': 'Minting', 'to': 'GhoAToken', 'amount': 3054},
 {'from': 'Minting', 'to': 'UniswapV3Pool', 'amount': 2574},
 {'from': 'Minting', 'to': 'ParaSwap', 'amount': 1514},
 {'from': 'Minting', 'to': 'GHO Staking', 'amount': 1317},
 {'from': 'Minting', 'to': 'CowSwap', 'amount': 1229},
 {'from': 'Minting', 'to': 'Others', 'amount': 112},
 {'from': 'Minting', 'to': '1inch', 'amount': 922},
 {'from': 'Minting', 'to': 'DeFi Saver', 'amount': 852},
 {'from': 'Minting', 'to': 'Curve', 'amount': 289},
 {'from': 'Minting', 'to': 'Balancer', 'amount': 260},
 {'from': 'Minting', 'to': 'AirSwap', 'amount': 212},
 {'from': 'Minting', 'to': 'Maverick', 'amount': 189},
 {'from': 'Minting', 'to': 'MEV Bots', 'amount': 81},
 {'from': 'Minting', 'to': '0x Exchange', 'amount': 79},
 {'from': 'Minting', 'to': 'GHO CCIP', 'amount': 70},
 {'from': 'Minting', 'to': 'Odos', 'amount': 64},
 {'from': 'Minting', 'to': 'Li.Fi', 'amount': 62},
 {'from': 'Minting', 'to': 'Kyber Network', 'amount': 58},
 {'from':

In [33]:
second_order = []
for protocol, amount in first_order_flows.items():
    if protocol not in dexes:
        continue
    for token in grouped.loc[protocol].sort_values('percentage', ascending=False).index:
        second_order.append(
            {'from': protocol,
            'to': token,
            'amount': int(grouped.loc[protocol,token].percentage / 100 * amount)}
        )
second_order

[{'from': 'UniswapV3Pool', 'to': 'USDC', 'amount': 943},
 {'from': 'UniswapV3Pool', 'to': 'USDT', 'amount': 412},
 {'from': 'UniswapV3Pool', 'to': 'BTC', 'amount': 212},
 {'from': 'UniswapV3Pool', 'to': 'Other Cryptos', 'amount': 196},
 {'from': 'UniswapV3Pool', 'to': 'ETH/LSTs/LRTs', 'amount': 188},
 {'from': 'UniswapV3Pool', 'to': 'AAVE', 'amount': 162},
 {'from': 'UniswapV3Pool', 'to': 'DAI', 'amount': 147},
 {'from': 'UniswapV3Pool', 'to': 'Providing Liquidity', 'amount': 134},
 {'from': 'UniswapV3Pool', 'to': 'LINK', 'amount': 97},
 {'from': 'UniswapV3Pool', 'to': 'Other Stablecoins', 'amount': 30},
 {'from': 'UniswapV3Pool', 'to': 'GHO', 'amount': 18},
 {'from': 'UniswapV3Pool', 'to': 'LUSD', 'amount': 15},
 {'from': 'UniswapV3Pool', 'to': 'Variable Debt Tokens', 'amount': 6},
 {'from': 'UniswapV3Pool', 'to': 'aTokens', 'amount': 6},
 {'from': 'ParaSwap', 'to': 'GHO', 'amount': 970},
 {'from': 'ParaSwap', 'to': 'USDC', 'amount': 218},
 {'from': 'ParaSwap', 'to': 'USDT', 'amount':

In [34]:
label_df.drop_duplicates(['from', 'to'])['symbol'].value_counts().to_list()
grouped.loc['UniswapV3Pool'].sort_values('percentage', ascending=False)

,count,percentage
symbol,,
USDC,715,36.647873
USDT,313,16.043055
BTC,161,8.252178
Other Cryptos,149,7.637109
ETH/LSTs/LRTs,143,7.329575
AAVE,123,6.304459
DAI,112,5.740646
Providing Liquidity,102,5.228088
LINK,74,3.792927


In [36]:
import json

with open('first_order_dup.json', 'w') as f:
    json.dump(sorted(first_order, key=lambda x: x['amount'], reverse=True), f, indent=2)

with open('second_order_dup.json', 'w') as f:
    json.dump(second_order, f, indent=2)

In [39]:
with open('first_order.json', 'r') as f:
    first_order = json.load(f)

with open('second_order.json', 'r') as f:
    second_order = json.load(f)

In [47]:
import plotly.graph_objects as go
import pandas as pd

def create_sankey(first_order, second_order):
    nodes = {}
    links = []
    
    def add_node(name):
        if name not in nodes:
            nodes[name] = len(nodes)
        return nodes[name]
    
    for item in first_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    for item in second_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    node_list = sorted(nodes.items(), key=lambda x: x[1])
    node_names = [name for name, _ in node_list]
    
    link_sources, link_targets, link_values = zip(*links)
    
    sankey = go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_names,
            color="blue"
        ),
        link=dict(
            source=link_sources,
            target=link_targets,
            value=link_values
        )
    )
    
    layout = go.Layout(
        title_text="Stablecoin Flow Sankey Diagram",
        font_size=10
    )
    
    fig = go.Figure(data=[sankey], layout=layout)
    return fig

sankey_fig = create_sankey(first_order, second_order)

sankey_fig.write_html("sankey_plot.html")
from IPython.display import IFrame
IFrame(src="sankey_plot.html", width=900, height=600)

In [35]:
import plotly.graph_objects as go

def create_sankey(first_order, second_order):
    nodes = {}
    links = []
    
    def add_node(name):
        if name not in nodes:
            nodes[name] = len(nodes)
        return nodes[name]
    
    for item in first_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    for item in second_order:
        source = add_node(item['from'])
        target = add_node(item['to'])
        links.append((source, target, item['amount']))
    
    node_list = sorted(nodes.items(), key=lambda x: x[1])
    node_names = [name for name, _ in node_list]
    
    link_sources, link_targets, link_values = zip(*links)
    
    sankey = go.Sankey(
        arrangement='snap',
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_names,
            color="blue",
            align='left'
        ),
        link=dict(
            source=link_sources,
            target=link_targets,
            value=link_values,
            arrowlen=15
        )
    )
    
    layout = go.Layout(
        title_text="Stablecoin Flow Sankey Diagram",
        font_size=10
    )
    
    fig = go.Figure(data=[sankey], layout=layout)
    return fig

sankey_fig = create_sankey(first_order, second_order)

sankey_fig.write_html("sankey_plot.html")
from IPython.display import IFrame
IFrame(src="sankey_plot.html", width=900, height=600)